# Tracing in OpenGGCM fields

In [ ]:
from __future__ import annotations

import numpy as np
import pyvista as pv
import xarray as xr
from scipy import constants

import ggcmpy.tracing


def to_mesh_lines(df):
    positions = df[["x", "y", "z"]].values
    mesh = pv.PolyData(positions)
    lines = pv.lines_from_points(positions)
    return mesh, lines


def plot_trajectory(plotter, df, **kwargs):
    _, lines = to_mesh_lines(df)
    plotter.add_mesh(lines, **kwargs)

### Load an actual OpenGGCM dataset

It's not from an actually meaningful simulation, but it'll do for now. The code below loads the data (which was generated with `etajout=true`), and then rescales to base SI units (FIXME: we should have the option to just trace in the original units).

In addition, it sets the electric field to zero to make things simpler.

In [ ]:
R_E = 6.371e6  # m

ds = xr.open_dataset(ggcmpy.sample_dir / "test0008.3df.000007")
x, y, z = R_E * ds.x.values, R_E * ds.y.values, R_E * ds.z.values
ds = ds.assign_coords(
    x=("x", x),
    y=("y", y),
    z=("z", z),
    x_nc=("x_nc", 0.5 * (x[1:] + x[:-1])),
    y_nc=("y_nc", 0.5 * (y[1:] + y[:-1])),
    z_nc=("z_nc", 0.5 * (z[1:] + z[:-1])),
)
ds["bx1"] = (("x_nc", "y", "z"), 1e-9 * ds.bx1.values[:-1, :, :])
ds["by1"] = (("x", "y_nc", "z"), 1e-9 * ds.by1.values[:, :-1, :])
ds["bz1"] = (("x", "y", "z_nc"), 1e-9 * ds.bz1.values[:, :, :-1])
ds["ex1"] = (("x", "y_nc", "z_nc"), 0 * ds.eflx.values[:, :-1, :-1])
ds["ey1"] = (("x_nc", "y", "z_nc"), 0 * ds.efly.values[:-1, :, :-1])
ds["ez1"] = (("x_nc", "y_nc", "z"), 0 * ds.eflz.values[:-1, :-1, :])
ds

### Trace using the Boris integrator

That's pretty much just the same as in the dipole example, but using the OpenGGCM fields.

In [ ]:
boris = ggcmpy.tracing.BorisIntegrator_f2py(ds, q=-constants.e, m=constants.m_e)
x0 = np.array([-5.0 * R_E, 0, 0])

B_x0 = boris._interpolator.B(x0)
T_e = 1.0 * 1e3 * constants.e  # 1 keV in J
v_e = np.sqrt(2 * T_e / constants.m_e)  # electron thermal speed
v_e *= 100.0

om_ce = np.abs(constants.e) * np.linalg.norm(B_x0) / constants.m_e  # gyrofrequency
r_ce = constants.m_e * v_e / (constants.e * np.linalg.norm(B_x0))  # gyroradius
print(f"B={B_x0} om_ce={om_ce} r_ce={r_ce}")  # noqa: T201

v0 = np.array([0.0, v_e, v_e])  # [m/s]
dt = 2 * np.pi / om_ce / 20
t_max = 500 * 2 * np.pi / om_ce  # [s]

df = boris.integrate(x0, v0, t_max, dt)
df

### Plot the trajectory

Not all that exciting, but at least it looks reasonable...

In [ ]:
plotter = pv.Plotter()
plot_trajectory(plotter, df, line_width=1, color="blue")
plotter.show()

### Check energy conservation

Looks pretty good -- the tracing here is done in Fortran with single precision, so a relative error of $10^{-6}$ is expected due to machine precision.

In [ ]:
df["E"] = 0.5 * constants.m_e * np.linalg.norm(df[["vx", "vy", "vz"]].values, axis=1)
df.plot(x="time", y="E", title="Energy of the particle over time");